In [ ]:
Day 7 / 爬虫综合进阶训练
    复习：将上上节课的小爬虫 改写为基于 selector 的版本
    爬取非文本数据
        获取图片链接
            img 元素
            src 属性
        下载图片
            发送请求
            储存图片到本地文件
                with open() as f
                f.write
            储存其它信息到本地文件
            读取本地文件
        下载音乐
    复杂逻辑
        检索
            search params 拼接
        换页
        获取身份：Cookie
    爬取结果的保存和读取
        保存到 XML
            读取
        保存到 JSON
            读取
        保存到 pickle
            读取

In [18]:
import requests
from html import unescape

# 目标：提取出所有的 HTML 标签和描述，放入一个 Python dictionary 数据结构。

# 第一步：获取网页的完整源码

响应 = requests.get("https://www.w3school.com.cn/tags/index.asp")
html = 响应.text

# 第二步：找到合适的特征字符串，进行逐步切割

# 特征字符串 1：<table class="dataintable">

# 定位
feature_string_1_position = html.index("<table class=\"dataintable\">")
# 切割
html_step_1 = html[feature_string_1_position + len("<table class=\"dataintable\">"):]


# 特征字符串 2：</table>

# 定位 index
feature_string_2_position = html_step_1.index("</table>")  # 注意要用 html_step_1 而不是 html
# 切割 [start:end]
html_step_2 = html_step_1[:feature_string_2_position]

# "abcdef".index("cd") == 2
# "abcdef"[2:] == "cdef"
# "abcdef"[2 + len("cd"):] == "ef"

# 第三步：已经切出了有用的部分，进行 split 分割成小单元处理，每个小单元的结构是一样的

# 用 split 分割成 N+1 个部分，得到一个列表
all_cells = html_step_2.split("</tr>")
# 去掉表头
all_cells.pop(0)

# 用于储存结果的列表变量
result_list = []

# 在循环里面分析每个小单元
for cell in all_cells:
    try:
        # 储存小单元分析结果的字典变量
        cell_data = {}

        # 通过 print 发现前 23 个字符没用，丢掉
        useful_data = cell[23:]
        # 通过 print 发现网址结尾之后立马是一个双引号字符
        link = useful_data[0:useful_data.index('"')]
        # 储存
        cell_data["href"] = link

        # 通过 print 发现元素名部分从第一个双引号字符往后两个字符开始，到第一个 </td> 结束
        # 定位+切割一步完成
        element_name = useful_data[useful_data.index('"') + 2 : useful_data.index('</td>')]
        # 删除两端空白符避免干扰（strip()），删除可能存在的 &nbsp; 后缀避免干扰，删除一定存在的 </a> 后缀 （这些都没用）
        element_name = element_name.strip().removesuffix("&nbsp;").removesuffix("</a>")
        # 反转义，&lt; --> 小于号， &gt; --> 大于号
        element_name = unescape(element_name)
        # 储存
        cell_data["name"] = element_name

        # 通过 print 发现 元素用途从 useful_data 里的第一个 <td> 开始，先切一刀
        explanation = useful_data[useful_data.index("<td>") + len("<td>"):]
        # 通过 print 发现 元素用途到 explanation 里的第一个 </td> 结束，再切一刀
        explanation = explanation[:explanation.index("</td>")]
        # 储存
        cell_data["usage"] = explanation

        # 把当前 cell 的结果存进全局的结果列表里面
        result_list.append(cell_data)
    except:
        continue

# 遍历结果列表，打印
for info in result_list:
    print("标签：", info["name"])
    print("用途：", info["usage"])
    print("说明链接：", "https://www.w3school.com.cn{}".format(info["href"]))
    print()

In [17]:
import requests, bs4
from html import unescape

# 目标：提取出所有的 HTML 标签和描述，放入一个 Python dictionary 数据结构。

# 第一步：获取网页的完整源码

响应 = requests.get("https://www.w3school.com.cn/tags/index.asp")
html = 响应.text

# 第二步：使用 bs4 完成树状结构的解析

soup = bs4.BeautifulSoup(html)
# print(soup.prettify())

tags = soup.select("#maincontent table.dataintable td:nth-child(1)")
usages = soup.select("#maincontent table.dataintable td:nth-child(2)")

assert len(tags) == len(usages)
assert len(tags) > 0

# 第三步：提取信息

result_list = []

for i in range(len(tags)):
    tag = tags[i]
    usage = usages[i]
    tag_links = tag.select("a")
    if len(tag_links) > 0:
        link = tag_links[0].get("href")
        result_list.append({ "name": tag.get_text(), "usage": usage.get_text(), "href": link })
    else:
        result_list.append({ "name": tag.get_text(), "usage": usage.get_text(), "href": "" })

# 遍历结果列表，打印

for info in result_list:
    print("标签：", info["name"])
    print("用途：", info["usage"])
    print("说明链接：", "https://www.w3school.com.cn{}".format(info["href"]) if info["href"] else "")
    print()

### 爬取非文本数据

#### 获取图片链接

    img 元素 https://www.w3school.com.cn/tags/tag_img.asp

```html
<img src="/i/eg_tulip.jpg"  alt="上海鲜花港 - 郁金香" />
```
    
    src 属性 和 alt 属性

#### 下载图片
    
    发送请求

```python
import requests

url = "https://www.w3school.com.cn/i/eg_tulip.jpg"
resp = requests.get(url)

if response.status_code == 200:
    print(resp.content)  # 图片不是文本，不能使用 .text
```
    
    储存信息到本地文件

        open() 和 f.close()

        打开文件的模式

1. Read Only (`r`) : Open text file for reading. The handle is positioned at the beginning of the file. If the file does not exists, raises the I/O error. This is also the default mode in which a file is opened.
2. Read and Write (`r+`): Open the file for reading and writing. The handle is positioned at the beginning of the file. Raises I/O error if the file does not exist.
3. Write Only (`w`) : Open the file for writing. For the existing files, the data is truncated and over-written. The handle is positioned at the beginning of the file. Creates the file if the file does not exist.
4. Write and Read (`w+`) : Open the file for reading and writing. For an existing file, data is truncated and over-written. The handle is positioned at the beginning of the file.
5. Append Only (`a`): Open the file for writing. The file is created if it does not exist. The handle is positioned at the end of the file. The data being written will be inserted at the end, after the existing data.
6. Append and Read (`a+`) : Open the file for reading and writing. The file is created if it does not exist. The handle is positioned at the end of the file. The data being written will be inserted at the end, after the existing data.

        f.write
    
        with open() as f

```python
file = open("./example.txt", 'w')
file.write("Hello")
file.close()
```

```python
file = open("./example.txt", 'a')
file.write(" World")
file.close()
```

```python
with open("./example.txt", 'w') as f:
    file.write("Hello")
```
    
    读取本地文件

        f.read(byte_count)

        f.readlines()

#### 本地文件路径的规则

在Windows和MacOS中，路径分隔符有所不同。

- Windows：在Windows中，路径分隔符使用反斜杠（`\`）。
- MacOS：在MacOS中，路径分隔符使用正斜杠（`/`）。

在Windows中，反斜杠（`\`）被用作路径分隔符，但它也是转义字符。因此，如果要在 Python 中使用字符串表示路径，并且要在路径中使用反斜杠本身，需要进行转义，即使用两个反斜杠（`\\`）表示一个反斜杠。MacOS则无需进行此操作。

例如，在Windows中，要使用 Python 表示路径`C:\Users\Username\Documents\File.txt`，需要编写如下代码：

```python
"C:\\Users\\Username\\Documents\\File.txt"
```

在MacOS中，要使用 Python 表示路径`/Users/Username/Documents/File.txt`：

```python
"/Users/Username/Documents/File.txt"
```

绝对路径：绝对路径是从硬盘根目录开始的完整路径。在Windows系统中，硬盘根目录使用驱动器号表示（例如 `C:\`、`D:\` 等）。在 MacOS 系统中，根目录的路径只有一个 `/` 字符。

相对路径：相对路径是相对于当前工作目录或指定的基准路径的路径。当前工作目录是指在命令提示符或脚本中执行命令时所处的目录。

`D:\Folder\File.txt` 为绝对路径。

相对路径需要有基准路径才能完成「相对」的操作。比如说当前 Jupyter Notebook 已经在某个目录下打开，那么可以相对这个目录，进行相对路径的编写和解析。

获取当前 Jupyter Notebook 路径的两种方法：

1. 通用方法

In [ ]:
import os

current_directory = os.getcwd()
print(current_directory)

2. 区分 MacOS 和 Windows 的方法

Windows（以下两行任选一行就行）

In [ ]:
!echo %cd%
!cd

MacOS:

In [ ]:
!pwd

Windows 上面相对路径的例子：

- 如果当前工作目录为C:\Users\Username\Documents，则Documents\File.txt 表示当前工作目录中的Documents文件夹中的File.txt文件。
- 如果当前工作目录为C:\Users\Username，则Documents\File.txt 表示当前工作目录中的Username文件夹中的Documents文件夹中的File.txt文件。

MacOS 上面相对路径的例子：

- 如果当前工作目录为 /Users/Username/Documents，则 File.txt 表示当前工作目录中的 File.txt 文件。
- 如果当前工作目录为 /Users/Username，则 Documents/File.txt 表示当前工作目录中的 Documents 文件夹中的 File.txt 文件。

换言之，它们实际上对应到同一个文件。

#### `.` 和 `..`

在文件路径中，`.` 和 `..` 表示当前目录和上一级目录，它们通常用于相对路径，但也可以用于绝对路径。

- `.`（单个点）：表示当前目录。当你在路径中使用 `.` 时，它指代的是当前所在的目录本身。
- `..`（两个点）：表示上一级目录。当你在路径中使用 `..` 时，它会使得当前路径返回上一级目录。

例子

- 如果当前工作目录为 /Users/A，则 `./File.txt` 表示当前工作目录中的 File.txt 文件 --> `/Users/A/File.txt`
- 如果当前工作目录为 /Users/A，则 `../File.txt` 表示当前工作目录的父级目录中的 File.txt 文件 --> `/Users/File.txt`


#### 下载音乐

    网易云音乐

    浏览器的 Network 工具：分析网络请求

    提取出音频文件链接

    使用 requests.get 下载

    wb 模式保存

In [20]:
import requests
headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36' }

resp = requests.get("https://m701.music.126.net/20230807174420/50355c2c0c2a5646f39054f05bcd89d1/jdyyaac/obj/w5rDlsOJwrLDjj7CmsOj/29233433335/7abf/8176/6024/f11013196255be4dec36cf8950b92266.m4a", headers=headers)

with open("./music.m4a", "wb") as f:
    f.write(resp.content)

#### 复杂逻辑

    检索

        search params 拼接

        requests.get 的 params 参数

##### 换页

##### 获取身份：Cookie

Cookie是由服务器发送到用户浏览器并存储在用户计算机上的小型文本文件。它用于在Web应用程序中跟踪和存储有关用户的信息。

当用户访问一个网站时，服务器可以通过HTTP响应头中的`Set-Cookie`标头将Cookie发送给用户的浏览器。浏览器接收到Cookie后，会将其存储在用户的计算机上。每当用户再次访问同一网站时，浏览器会通过HTTP请求头中的`Cookie`标头将存储的Cookie信息发送回服务器。

Cookie通常用于以下目的：

- 身份认证和会话管理：通过将唯一的会话标识符存储在Cookie中，服务器可以跟踪用户的认证状态。服务器可以通过 Cookie 获知用户的登录状态。
- 用户偏好设置：Cookie可以用于保存用户的偏好设置，如语言偏好、主题选择等。
- 跟踪和分析：通过在Cookie中存储跟踪代码或标识符，网站可以分析用户的行为，如页面浏览量、点击量等。
- 广告定向：广告商可以使用Cookie来跟踪用户的兴趣和行为，以便提供个性化的广告体验。

注意：Cookie是存储在用户计算机上的文本文件，因此可以被用户主动删除

#### 爬取结果的保存和读取

    保存到 XML - 使用 xmltodict（需要安装）

        读取

    保存到 JSON - json.dumps

        读取 - json.loads

    保存到 pickle - pickle.dumps

        读取 - pickle.loads

        pickle 可以保存任意变量

In [23]:
!pip install xmltodict


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


从 dict 到 xml 字符串：

In [24]:
import xmltodict
print(xmltodict.unparse({ 'a': 1 }))

<?xml version="1.0" encoding="utf-8"?>
<a>1</a>


从 xml 到 dict：

In [25]:
import xmltodict
print(xmltodict.parse('<?xml version="1.0" encoding="utf-8"?><a>1</a>'))

{'a': '1'}
